# Import things

In [1]:
import os
import difflib
import openai   
from dotenv import load_dotenv  

from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool

In [2]:
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [3]:
# 1. Mock Supply Chain Data
transport_data = {
    "Transport1": {"Paletten": 1000, "Rahmen": 100},
    "Transport2": {"Paletten": 850,  "Rahmen": 2},
    "Transport3": {"Paletten": 920,  "Rahmen": 20},
    "Transport4": {"Paletten": 1100, "Rahmen": 1},
    "Transport5": {"Paletten": 980,  "Rahmen": 4}
}

# Define Tools

In [4]:
@tool
def suggest_company_name(input_company: str) -> str:
    """
    Suggests the closest matching company name from the available companies.

    Uses fuzzy matching (via difflib.get_close_matches) to determine if the input_company
    is similar to any of the companies in transport_data.

    :param input_company: The company name provided by the user, potentially with a typo.
    :return: A suggestion message such as "Did you mean 'Transport2'?" or an indication
             that no similar company was found.
    """
    valid_companies = list(transport_data.keys())
    matches = difflib.get_close_matches(input_company, valid_companies, n=1, cutoff=0.6)
    if matches:
        # If the best match is identical (ignoring case), no suggestion is needed.
        if matches[0].lower() == input_company.lower():
            return f"'{input_company}' is recognized."
        else:
            return f"Did you mean '{matches[0]}'?"
    else:
        return f"No similar company name found for '{input_company}'. Please verify the company name."


In [5]:
@tool
def call_suggestion_agent(query: str) -> str:
    """
    The coordinator calls suggestionAgent in single-step usage for inventory queries.
    """
    return "[Coordinator] (No real implementation yet)"

In [6]:
def create_company_suggestion_agent():
    llm_suggester = ChatOpenAI(model="gpt-4o", temperature=0)
    company_suggestion_agent = create_react_agent(
        model=llm_suggester,
        tools=[suggest_company_name]
    )
    return company_suggestion_agent

In [7]:
company_suggestion_agent = create_company_suggestion_agent()


In [8]:
def call_suggestion_agent_impl(query: str) -> str:
    """
    Actually calls Suggestion Agent in single-step usage.
        """
    SYSTEM_MSG_DELIVERY = """
    You are the CompanySuggestionAgent. Help suggest the closest matching company name based on user input.

    """
    result = company_suggestion_agent.invoke(
        {
            "messages": [
                {"role": "system", "content": SYSTEM_MSG_DELIVERY},
                {"role": "user", "content": query}
            ]
        },
        config={"configurable": {"recursion_limit": 50}}
    )
    return result["messages"][-1].content

In [9]:
call_suggestion_agent.__doc__ = call_suggestion_agent_impl.__doc__
call_suggestion_agent.func = call_suggestion_agent_impl

## Stock tools

In [10]:
@tool
def check_container(item: str, company: str = None) -> str:
    """
    Checks the container stock for the specified item.

    If a company is provided, the tool returns the container stock for that company.
    If the provided company name does not exactly match a known company (even in a case-insensitive check),
    the tool delegates to the CompanySuggestionAgent to see if a close match can be found.
    If no company is provided, the tool aggregates the container stock across all companies.

    :param item: e.g. "Paletten" or "Rahmen"
    :param company: Optional. The specific company to check.
    :return: A message indicating the container stock or a suggestion if the company name is not recognized.
    """
    if company:
        # Build a mapping of lowercase company names to the actual keys in transport_data.
        company_mapping = {key.lower(): key for key in transport_data.keys()}
        company_lower = company.lower()
        if company_lower in company_mapping:
            actual_company = company_mapping[company_lower]
            qty = transport_data[actual_company].get(item, 0)
            return f"[balance] '{item}' at {actual_company}: {qty} units."
        else:
            # Delegate to the CompanySuggestionAgent if the company isn't found.
            response = company_suggestion_agent.invoke(
                {
                    "messages": [
                        {"role": "system", "content": "You are the CompanySuggestionAgent. Help suggest the closest matching company name based on user input."},
                        {"role": "user", "content": company}
                    ]
                },
                config={"configurable": {"recursion_limit": 10}}
            )
            return response["messages"][-1].content
    else:
        total_qty = sum(comp.get(item, 0) for comp in transport_data.values())
        return f"[balance] '{item}' (all companies): {total_qty} units."


In [11]:
@tool
def update_container(item_qty: str, company: str = "Transport1") -> str:
    """
    Updates container stock for a specified transport company by adding or subtracting a given quantity.

    :param item_qty: e.g. "add 100, Paletten" or "subtract 5, Rahmen"
    :param company: The transport company to update (default is "Transport1")
    :return: Confirmation or error message.
    """
    try:
        parts = item_qty.split(",")  # Ensure correct parsing
        if len(parts) != 2:
            return "[balance] Update error: Invalid format. Use 'add X, item' or 'subtract X, item'."

        operation_with_qty, item = parts
        operation_parts = operation_with_qty.strip().split()

        if len(operation_parts) != 2:
            return "[balance] Update error: Invalid format. Expected 'add X' or 'subtract X'."

        operation, qty_str = operation_parts
        qty = int(qty_str.strip())  # Convert quantity to an integer
        item = item.strip()  # Remove leading/trailing spaces

        if company not in transport_data:
            return f"[balance] Update error: Company '{company}' not found."

        if operation.lower() == "add":
            transport_data[company][item] = transport_data[company].get(item, 0) + qty
        elif operation.lower() == "subtract":
            transport_data[company][item] = max(0, transport_data[company].get(item, 0) - qty)
        else:
            return f"[balance] Update error: Unknown operation '{operation}'. Use 'add' or 'subtract'."

        return f"[balance] Successfully {operation}ed {qty} units of '{item}' to {company}."

    except Exception as e:
        return f"[balance] Update error: {e}"


## Tool to agents to cooperate

In [12]:
@tool
def call_suggestion_agent(query: str) -> str:
    """
    The coordinator calls suggestionAgent in single-step usage for inventory queries.
    """
    return "[Coordinator] (No real implementation yet)"

In [13]:
@tool
def call_container_agent(query: str) -> str:
    """
    The coordinator calls ContainerAgent in single-step usage for inventory queries.
    """
    return "[Coordinator] (No real implementation yet)"

# Create agents

In [14]:
def create_specialized_agents():
    llm_stock = ChatOpenAI(model="gpt-4o", temperature=0)

    # StockAgent
    container_agent = create_react_agent(
        model=llm_stock,
        tools=[update_container, check_container]
    )

    return container_agent

In [15]:
def create_company_suggestion_agent():
    llm_suggester = ChatOpenAI(model="gpt-4o", temperature=0)
    company_suggestion_agent = create_react_agent(
        model=llm_suggester,
        tools=[suggest_company_name]
    )
    return company_suggestion_agent

In [16]:
def create_coordinating_agent():
    # We'll do a ReAct agent that can call call container agent, etc.
    llm_coord = ChatOpenAI(model="gpt-4o", temperature=0)
    coordinating_agent = create_react_agent(
        model=llm_coord,
        tools=[call_container_agent]
    )
    return coordinating_agent

# Use our agents

In [17]:
# Create specialized agents (Stock, Delivery, Priority)
container_agent = create_specialized_agents()


# Create the coordinating agent
coordinating_agent = create_coordinating_agent()

In [18]:
def call_container_agent_impl(query: str) -> str:
    """
    Actually calls StockAgent in single-step usage.
    """
    SYSTEM_MSG_STOCK = """
    You are the StockAgent. 
    You have 'check_stock(item)' and 'update_stock(item_qty)'.
    Call exactly one of them based on the user's query. Return the tool's output.
    """
    result = container_agent.invoke(
        {
            "messages": [
                {"role": "system", "content": SYSTEM_MSG_STOCK},
                {"role": "user",   "content": query}
            ]
        },
        config={"configurable": {"recursion_limit": 50}}
    )
    return result["messages"][-1].content

In [19]:
def call_suggestion_agent_impl(query: str) -> str:
    """
    Actually calls Suggestion Agent in single-step usage.
        """
    SYSTEM_MSG_DELIVERY = """
    You are the CompanySuggestionAgent. Help suggest the closest matching company name based on user input.

    """
    result = company_suggestion_agent.invoke(
        {
            "messages": [
                {"role": "system", "content": SYSTEM_MSG_DELIVERY},
                {"role": "user", "content": query}
            ]
        },
        config={"configurable": {"recursion_limit": 50}}
    )
    return result["messages"][-1].content

In [20]:
# Patch docstrings & function references onto the existing 'call_*' tools
call_container_agent.__doc__ = call_container_agent_impl.__doc__
call_container_agent.func = call_container_agent_impl

call_suggestion_agent.__doc__ = call_suggestion_agent_impl.__doc__
call_suggestion_agent.func = call_suggestion_agent_impl

## Step 7: Create the Coordinating Agent


In [21]:

COORDINATOR_SYSTEM_MSG = """
You are the CoordinatingAgent for a supply chain.
Your job is to process user queries regarding container stock and defective containers.
If a user's query contains a company name that appears to be misspelled (for example, if a sub-agent returns a suggestion such as "Did you mean 'Transport2'?"),
ask the user for clarification. For instance, you might say: "It looks like you might have meant 'Transport2'. Is that correct?"
Don't wait for the user's confirmation and proceede anyway.
When the query is clear, return the final result without any disclaimers.
"""

llm_coord = ChatOpenAI(model="gpt-4o", temperature=0)
coordinating_agent = create_react_agent(
    model=llm_coord,
    tools=[call_container_agent, call_suggestion_agent_impl],
)


## Step 8: Set Up Chat History Accumulator

In [23]:
chat_history = [{"role": "system", "content": COORDINATOR_SYSTEM_MSG}]

def add_to_history(role: str, content: str):
    """
    Helper function to add a new message to the chat history.
    
    :param role: "user" or "assistant"
    :param content: The message content.
    """
    chat_history.append({"role": role, "content": content})


## Step: Testing the Chatbot

In [24]:
# Step 1: User submits the initial query.
initial_query = "Please let me know the current stock of Rahmen Containers trnasport1"
add_to_history("user", initial_query)
print("User:",initial_query)


response1 = coordinating_agent.invoke(
    {"messages": chat_history},
    config={"configurable": {"recursion_limit": 80}}
)
agent_reply1 = response1["messages"][-1].content
add_to_history("assistant", agent_reply1)

print("CoordinatingAgent:", agent_reply1)



# Step 3: Adding stock.
user_question2 = "Please add 2 Rahmen to Transport3"
add_to_history("user", user_question2)
print("\nUser:", user_question2)

response3 = coordinating_agent.invoke(
    {"messages": chat_history},
    config={"configurable": {"recursion_limit": 80}}
)
agent_reply3 = response3["messages"][-1].content
add_to_history("assistant", agent_reply3)

print("CoordinatingAgent:", agent_reply3)


User: Please let me know the current stock of Rahmen Containers trnasport1
CoordinatingAgent: The current stock of Rahmen Containers at Transport1 is 100 units.

User: Please add 2 Rahmen to Transport3
CoordinatingAgent: 2 units of 'Rahmen' have been successfully added to Transport3.


In [25]:

COORDINATOR_SYSTEM_MSG = """
You are the CoordinatingAgent for a supply chain.
Your job is to process user queries regarding container stock and defective containers.
If a user's query contains a company name that appears to be misspelled (for example, if a sub-agent returns a suggestion such as "Did you mean 'Transport2'?"),
ask the user for clarification. For instance, you might say: "It looks like you might have meant 'Transport2'. Is that correct?"

Wait for the users confirmation and proceed after.

When the query is clear, return the final result without any disclaimers.
"""

llm_coord = ChatOpenAI(model="gpt-4o", temperature=0)
coordinating_agent = create_react_agent(
    model=llm_coord,
    tools=[call_container_agent, call_suggestion_agent_impl],
)


In [26]:
# Step 1: User submits the initial query.
initial_query = "Please let me know the current stock of Rahmen Containers trnasport1"
add_to_history("user", initial_query)
print("User:",initial_query)


response1 = coordinating_agent.invoke(
    {"messages": chat_history},
    config={"configurable": {"recursion_limit": 80}}
)
agent_reply1 = response1["messages"][-1].content
add_to_history("assistant", agent_reply1)

print("CoordinatingAgent:", agent_reply1)

# Step 2: The agent asks for clarification and the user responds.
user_confirmation = "Yes"
add_to_history("user", user_confirmation)
print("\nUser:", user_confirmation)

response2 = coordinating_agent.invoke(
    {"messages": chat_history},
    config={"configurable": {"recursion_limit": 80}}
)
agent_reply2 = response2["messages"][-1].content
add_to_history("assistant", agent_reply2)

print("CoordinatingAgent:", agent_reply2)

# Step 3: Adding stock.
user_question2 = "Please add 2 Rahmen to Transport3?"
add_to_history("user", user_question2)
print("\nUser:", user_question2)

response3 = coordinating_agent.invoke(
    {"messages": chat_history},
    config={"configurable": {"recursion_limit": 80}}
)
agent_reply3 = response3["messages"][-1].content
add_to_history("assistant", agent_reply3)

print("CoordinatingAgent:", agent_reply3)


User: Please let me know the current stock of Rahmen Containers trnasport1
CoordinatingAgent: It looks like you might have meant 'Transport1'. The current stock of Rahmen Containers at Transport1 is 100 units.

User: Yes
CoordinatingAgent: The current stock of Rahmen Containers at Transport1 is 100 units.

User: Please add 2 Rahmen to Transport3?
CoordinatingAgent: 2 units of 'Rahmen' have been successfully added to Transport3.
